In [1]:
// nuget references
#r "nuget: FSharp.Stats, 0.5.1-preview.1"
#r "nuget: Plotly.NET.Interactive, 4.2.1"
#r "nuget: FSharp.Data, 6.3.0"
#r "nuget: Cytoscape.NET.Interactive, 0.2.0"

open FSharp.Stats
open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects
open FSharp.Data
open Cytoscape.NET
open System


//FSharp.Stats.ServiceLocator.setEnvironmentPathVariable (@"C:\Users\bvenn\source\repos\FSharp.Stats\lib")
//FSharp.Stats.Algebra.LinearAlgebra.Service()

//axis styling extension module
module Chart = 
    let myAxis name = LinearAxis.init(Title=Title.init name,Mirror=StyleParam.Mirror.All,Ticks=StyleParam.TickOptions.Inside,ShowGrid=false,ShowLine=true)
    let withAxisTitles x y chart = 
        chart 
        |> Chart.withTemplate ChartTemplates.lightMirrored
        |> Chart.withXAxis (myAxis x) 
        |> Chart.withYAxis (myAxis y)

type Category =
    | Beer
    | Beverage
    | Coffee
    | Other
    with 
        static member FromString (s: string) =
            match s with
            | "Beer" -> Beer
            | "Beverage" -> Beverage
            | "Coffee" -> Coffee
            | _ -> Other

type Order = {
    DateTime    : System.DateTime
    Name        : string
    Gender      : char
    Product     : string
    Price       : float
    Department  : string
    Category    : Category
    Amount      : int
    } with
        static member Create time (name: string) gender product price department category amount = {
            DateTime  = time
            Name      = name
            Gender    = gender
            Product   = product
            Price     = price
            Department= department
            Category  = category
            Amount    = amount
            }

let data = 
    let read =
        CsvFile
            //.Load(@"..\data\coffeedata.txt")
            .Load(@"C:\Users\bvenn\source\repos\brewing-discoveries-workshop\data\coffeedata.txt")
            .Cache()
    read.Rows
    |> Seq.map (fun row -> 
        Order.Create
            (System.DateTime.ParseExact((row.GetColumn "DateTime"),"dd/MM/yyyy HH:mm:ss",null))
            (row.GetColumn "Name")
            (row.GetColumn "Gender" |> char)
            (row.GetColumn "Product")
            (row.GetColumn "Price" |> float) 
            (row.GetColumn "Department")
            ((row.GetColumn "Category") |> Category.FromString)
            (row.GetColumn "Amount" |> int)
        )
    |> Array.ofSeq

let getDepartmentColor (department: string) = 
    match department with 
    | "Breakroom Bandits" -> "#2b3ae9"
    | "Genesis" -> "#f7da41"
    | "We Tried" -> "#008b66"
    | "No Lucks Given" -> "#987200"
    | "Toon Squad" -> "#ff7f0e"
    | "Rumor Spreaders" -> "#20b2aa"
    | "Risky Biscuits" -> "#a230ed"
    | "Recruitables" -> "#d21102"
    | "Employees of the Moment" -> "#19d3f3"
    | "Chargers" -> "#dea57b"
    | "Kickstarters" -> "#dea57b"
    | _ -> "#8b8b8b"

Installed Packages Cytoscape.NET.Interactive, 0.2.0 FSharp.Data, 6.3.0 FSharp.Stats, 0.5.1-preview.1 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\bvenn\.nuget\packages\plotly.net.interactive\4.2.1\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

Loading extensions from `C:\Users\bvenn\.nuget\packages\cytoscape.net.interactive\0.2.0\interactive-extensions\dotnet\Cytoscape.NET.Interactive.dll`

In [10]:

let getHistogramOfCategory (category: Category) = 
    data
    |> Array.filter (fun x -> x.Category = category && x.DateTime.DayOfWeek <> System.DayOfWeek.Saturday && x.DateTime.DayOfWeek <> System.DayOfWeek.Sunday)
    |> Array.groupBy (fun x -> x.DateTime.ToShortDateString())
    |> Array.map (snd >> Array.length)
    |> fun x -> 
        Chart.Histogram(x,Opacity=0.5,HistNorm=StyleParam.HistNorm.Probability)
        |> Chart.withTraceInfo (category.ToString() + " histogram")
        |> Chart.withSize(1000.,900.)

let histoRawBeer =  getHistogramOfCategory Category.Beer
let histoRawCoffe = getHistogramOfCategory Category.Coffee
let histoRawBeverage  = getHistogramOfCategory Category.Beverage

[
histoRawBeer 
histoRawCoffe
histoRawBeverage
] 
|> Chart.combine
|> Chart.withLayoutStyle(BarMode=StyleParam.BarMode.Overlay)
|> Chart.withAxisTitles "logs per work day" "probability density"

<!-- Plotly chart will be drawn inside this DIV -->

In [3]:
let persons = 
    data 
    |> Array.map (fun x -> x.Name,getDepartmentColor x.Department) 
    |> Array.distinct
    |> Map.ofArray

let searchDrinkPartners = 
    data
    |> Array.map (fun x -> 
        data
        |> Array.filter (fun t -> 
            x.DateTime < t.DateTime.AddMinutes 1 && x.DateTime > t.DateTime.AddMinutes -1
            )
        |> Array.map (fun drinkPartner -> 
            x.Name,drinkPartner.Name
            )
        |> Array.filter (fun (a,b) -> a <> b)
        )
    |> Array.concat
    |> Array.groupBy id
    |> Array.map (fun (partner,occurences) -> 
        partner,occurences.Length
        )
    |> Array.sortByDescending snd
    |> Array.filter (fun (a,b) -> b > 5)
    |> Array.distinctBy (fun ((a,b),_) -> [a;b] |> List.sort)

let getCytoVertices (input: ((string*string)*int) []) = 
    input
    |> Seq.collect (fun ((s,t),w) ->
        let stylingSource = [CyParam.label s; CyParam.weight 12; CyParam.color persons.[s]]
        let stylingTarget = [CyParam.label t; CyParam.weight 12; CyParam.color persons.[t]]
        [|Elements.node s stylingSource;Elements.node t stylingTarget|]
        )
    |> Seq.distinct

let getCytoEdges (input: seq<(string*string)*int>)= 
    input 
    |> Seq.distinct
    |> Seq.mapi (fun i ((s,t),w) -> 
        let styling = [CyParam.weight (sqrt(float w / 2.))]
        Elements.edge ("e" + string i) s t styling
        )

let goVertices = getCytoVertices searchDrinkPartners
let goEdges = getCytoEdges searchDrinkPartners


let cytoGraph vertices edges = 
    CyGraph.initEmpty ()
    |> CyGraph.withElements vertices
    |> CyGraph.withElements edges
    |> CyGraph.withStyle "node" 
        [
            CyParam.shape "circle"
            CyParam.content =. CyParam.label
            CyParam.Background.color  =.CyParam.color //"grey"//
            CyParam.Border.color "#A00975"
        ]
    |> CyGraph.withStyle "edge" 
        [
            CyParam.Line.color "#3D1244"
            CyParam.Curve.style "bezier"
            CyParam.width =. CyParam.weight
        ]
    |> CyGraph.withLayout (Layout.initCose id)   

cytoGraph goVertices goEdges
|> CyGraph.withSize (1300,1000)
|> CyGraph.show

In [6]:

let personsCorr = 
    persons
    |> Map.toArray
    |> Array.map fst

let signal = 
    data
    |> Array.groupBy (fun x -> x.DateTime.Date)
    |> Array.map (fun (date,orders) -> 
        
        let tmp = 
            personsCorr
            |> Array.map (fun name -> 
                orders
                |> Array.filter (fun order -> order.Name = name)
                |> Array.length
                |> float
                )
        tmp
        )
    |> JaggedArray.transpose

let corrMat = 
    Correlation.Matrix.rowWisePearson (matrix signal)

corrMat
|> Matrix.toJaggedArray
|> fun x -> Chart.Heatmap (x,colNames = personsCorr,rowNames=personsCorr)
|> Chart.withSize(1000.,900.)
|> Chart.show

corrMat
|> Matrix.toJaggedArray
|> Array.concat
|> Array.filter (fun x -> x <> 1.)
|> Chart.Histogram
|> Chart.withAxisTitles "pearson correlation" "count"
|> Chart.withSize(1000.,900.)
|> Chart.show

let correlationThreshold = 
    //precomputed because of runtime and LAPACK dependency
    //Testing.RMT.compute 0.9 0.01 0.05 corrMat
    0.333984375
    0.5

let mutable nodelist : string list= []
let mutable edgelist: (string*string*float) list= []

corrMat
|> Matrix.mapi (fun r c x -> 
    if r < c then 
        if x > 0.35547 then 
            nodelist <- personsCorr.[r]::(personsCorr.[c]::nodelist)
            edgelist <- (personsCorr.[r],personsCorr.[c],x)::edgelist
            1.
        else 0.
    else 0.)

let csbCytoVertices = 
    nodelist
    |> Seq.collect (fun s ->
        let stylingSource = [CyParam.label s; CyParam.weight 12; CyParam.color persons.[s]]
        [|Elements.node s stylingSource|]
        )
    |> Seq.distinct

let csbCytoEdges = 
    edgelist 
    |> Seq.distinct
    |> Seq.mapi (fun i (s,t,w) -> 
        let styling = [CyParam.weight (3. / abs w)]
        Elements.edge ("e" + string i) s t styling
        )

CyGraph.initEmpty ()
|> CyGraph.withElements csbCytoVertices
|> CyGraph.withElements csbCytoEdges
|> CyGraph.withStyle "node" 
    [
        CyParam.shape "circle"
        CyParam.content =. CyParam.label
        CyParam.Background.color  =.CyParam.color //"grey"//
        CyParam.Border.color "#A00975"
    ]
|> CyGraph.withStyle "edge" 
    [
        CyParam.Line.color "#3D1244"
        CyParam.Curve.style "bezier"
        CyParam.width =. CyParam.weight
    ]
|> CyGraph.withLayout (Layout.initCose id)  
|> CyGraph.withSize (1300,1000)
|> CyGraph.show
